In [17]:

import pandas as pd
import numpy as np

from ftfy import *
import nltk
from rake_nltk import Rake

from utils.visualization import *
from utils.datapreprocessing import *

import missingno as msno
import matplotlib.pyplot as plt 
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns 

# For Translation
from googletrans import Translator
from langdetect import detect

#For Upsampling
import imblearn
from collections import Counter
from matplotlib import pyplot
from numpy import where
# from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

# Import label encoder 
from sklearn import preprocessing 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Import Dependencies

import os
import re

import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import keras

from keras import layers
# For Embedding Layer
from keras.layers.embeddings import Embedding

# For RNN Layer
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Bidirectional
from keras.layers import Dense, Dropout, Activation, Input,  Flatten

# import numpy as np
# from tensorflow import keras
# from tensorflow.keras import layers

# For CNN Layer
from keras.layers import Conv1D, MaxPooling1D
import tensorflow as tf

# For Text Summarization
from gensim.summarization import summarize

In [18]:
filename= "dataset/final_dataframe_forDeep_Learning_Model.csv"
df = pd.read_csv(filename)

In [19]:
df.head(5)

,Complete_Description,Assignment group
0,event: critical:HostName_221.company.com the v...,GRP_1
1,"when undocking pc , screen will not come back ...",GRP_3
2,"received from: emailaddress gentles, i hav...",GRP_4
3,received from: emailaddress hi - the print...,GRP_5
4,received from: emailaddress job Job_1424 fail...,GRP_6


In [20]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['Assignment group'] = label_encoder.fit_transform(df['Assignment group']) 
  
df['Assignment group'].unique() 

array([ 8, 30, 41, 52, 63, 74, 79, 80,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36,
       37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 64, 65, 33, 66, 67, 68, 69, 70, 71, 72,
       73, 75, 76, 77, 78,  2,  3,  0,  1,  4,  6,  5,  7])

In [21]:
y = df['Assignment group']
y.shape

(52229,)

In [22]:
counter = Counter(y)
print(counter)

Counter({3: 769, 4: 752, 2: 741, 8: 661, 30: 661, 41: 661, 52: 661, 63: 661, 74: 661, 79: 661, 80: 661, 9: 661, 10: 661, 11: 661, 12: 661, 13: 661, 14: 661, 15: 661, 16: 661, 17: 661, 18: 661, 19: 661, 20: 661, 21: 661, 22: 661, 23: 661, 24: 661, 25: 661, 26: 661, 27: 661, 28: 661, 29: 661, 31: 661, 32: 661, 34: 661, 35: 661, 36: 661, 37: 661, 38: 661, 39: 661, 40: 661, 42: 661, 43: 661, 44: 661, 45: 661, 46: 661, 47: 661, 48: 661, 49: 661, 50: 661, 51: 661, 53: 661, 54: 661, 55: 661, 56: 661, 57: 661, 58: 661, 59: 661, 60: 661, 61: 661, 62: 661, 64: 661, 65: 661, 33: 661, 66: 661, 67: 661, 68: 661, 69: 661, 70: 661, 71: 661, 72: 661, 73: 661, 75: 661, 76: 661, 77: 661, 78: 661, 0: 557, 1: 413, 6: 301, 5: 267, 7: 176})


In [23]:
MAX_LENGTH = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Complete_Description.values)
post_seq = tokenizer.texts_to_sequences(df.Complete_Description.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.3, random_state=1)

In [25]:
X_train.shape,X_test.shape

((36560, 300), (15669, 300))

In [26]:
y_train.shape, y_test.shape

((36560,), (15669,))

In [27]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

16390

In [28]:
max_features = vocab_size  
maxlen = 200
num_class = len(np.unique(df['Assignment group'].values))

In [29]:
def call(self, h, mask=None):
    h_shape = K.shape(h)
    d_w, T = h_shape[0], h_shape[1]
    
    logits = K.dot(h, self.w)  # w^T h
    logits = K.reshape(logits, (d_w, T))
    alpha = K.exp(logits - K.max(logits, axis=-1, keepdims=True))  # exp
    alpha = alpha / K.sum(alpha, axis=1, keepdims=True)  # softmax
    r = K.sum(h * K.expand_dims(alpha), axis=1)  # r = h*alpha^T
    h_star = K.tanh(r)  # h^* = tanh(r)
    return h_star

In [30]:
class Attention(tf.keras.Model):
	def __init__(self, units):
		super(Attention, self).__init__()
		self.W1 = tf.keras.layers.Dense(units)
		self.W2 = tf.keras.layers.Dense(units)
		self.V = tf.keras.layers.Dense(1)

	def call(self, features, hidden):
		# hidden shape == (batch_size, hidden size)
		# hidden_with_time_axis shape == (batch_size, 1, hidden size)
		# we are doing this to perform addition to calculate the score
		hidden_with_time_axis = tf.expand_dims(hidden, 1)
		  
		# score shape == (batch_size, max_length, 1)
		# we get 1 at the last axis because we are applying score to self.V
		# the shape of the tensor before applying self.V is (batch_size, max_length, units)
		score = tf.nn.tanh(
			self.W1(features) + self.W2(hidden_with_time_axis))
		# attention_weights shape == (batch_size, max_length, 1)
		attention_weights = tf.nn.softmax(self.V(score), axis=1)
		  
		# context_vector shape after sum == (batch_size, hidden_size)
		context_vector = attention_weights * features
		context_vector = tf.reduce_sum(context_vector, axis=1)
		return context_vector, attention_weights

In [38]:
sequence_input = Input(shape=(None,), dtype="int32")
embedded_sequences = Embedding(max_features, 128)(sequence_input)
lstm = Bidirectional(LSTM(64, return_sequences = True), name="bi_lstm_0")(embedded_sequences)

# Getting our LSTM outputs
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(64, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])
context_vector, attention_weights = Attention(10)(lstm, state_h)
dropout = Dropout(0.2)(context_vector)
dense1 = Dense(32, activation="relu")(dropout)

output = Dense(num_class, activation="sigmoid")(dense1)
  
model = keras.Model(inputs=sequence_input, outputs=output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 128)    2097920     input_5[0][0]                    
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectional)       (None, None, 128)    98816       embedding_4[0][0]                
__________________________________________________________________________________________________
bi_lstm_1 (Bidirectional)       [(None, None, 128),  98816       bi_lstm_0[0][0]                  
____________________________________________________________________________________________

In [39]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

Epoch 1/7
1828/1828 [==============================] - 536s 293ms/step - loss: 2.2663 - acc: 0.4049 - val_loss: 0.8514 - val_acc: 0.7742
Epoch 2/7
1828/1828 [==============================] - 530s 290ms/step - loss: 0.6000 - acc: 0.8416 - val_loss: 0.3903 - val_acc: 0.8938
Epoch 3/7
1828/1828 [==============================] - 523s 286ms/step - loss: 0.3094 - acc: 0.9197 - val_loss: 0.3019 - val_acc: 0.9246
Epoch 4/7
1828/1828 [==============================] - 489s 268ms/step - loss: 0.2026 - acc: 0.9470 - val_loss: 0.2494 - val_acc: 0.9388
Epoch 5/7
1828/1828 [==============================] - 529s 289ms/step - loss: 0.1476 - acc: 0.9615 - val_loss: 0.2387 - val_acc: 0.9462
Epoch 6/7
1828/1828 [==============================] - 482s 264ms/step - loss: 0.1077 - acc: 0.9716 - val_loss: 0.2526 - val_acc: 0.9453
Epoch 7/7
1828/1828 [==============================] - 459s 251ms/step - loss: 0.0898 - acc: 0.9764 - val_loss: 0.2316 - val_acc: 0.9506
